<a href="https://colab.research.google.com/github/ayushi15/ayushi15/blob/main/NN-sigmoid%20activation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
data = pd.read_csv('USCensusTraining.csv')
data.shape

(25000, 15)

### Data prep/cleaning

In [3]:
data = data.replace('?',np.nan)

In [4]:
data.columns[data.isna().any()]

Index(['workclass', 'occupation', 'native-country'], dtype='object')

In [6]:
for column in data.columns:
  print(column + " missing values: " + str(data[column].isna().sum()))

age missing values: 0
workclass missing values: 1399
demogweight missing values: 0
education missing values: 0
education-num missing values: 0
marital-status missing values: 0
occupation missing values: 1404
relationship missing values: 0
race missing values: 0
sex missing values: 0
capital-gain missing values: 0
capital-loss missing values: 0
hours-per-week missing values: 0
native-country missing values: 445
income missing values: 0


In [7]:
#Excluding rows with NAs
df=data.dropna()

In [8]:
#Dropped 1829 rows with NAs
df.shape

(23171, 15)

## Part 1

In [9]:
pd.set_option('display.max_columns', None)
#Removing education-num and turning all categorical variables into dummy variables
no_edunum_df = df.drop(columns = ['education-num'])
#13 predictors, 1 response Y 'income'
cat_var = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'native-country']
df_dummy = pd.get_dummies(no_edunum_df, columns = cat_var)
# set binary values for 'sex' and 'income'
df_dummy['sex'] = df_dummy['sex'].replace('Male', 1)
df_dummy['sex'] = df_dummy['sex'].replace('Female', 0)
df_dummy['income'] = df_dummy['income'].replace('<=50K.', 0)
df_dummy['income'] = df_dummy['income'].replace('>50K.', 1)
df_dummy.shape

(23171, 103)

In [10]:
df_dummy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23171 entries, 0 to 24999
Columns: 103 entries, age to native-country_Yugoslavia
dtypes: int64(7), uint8(96)
memory usage: 3.5 MB


In [11]:
# standardize the dataset by using preprocessing to rescale the data
from sklearn import preprocessing 
scaler = preprocessing.MinMaxScaler()
scaled_dataset = pd.DataFrame(scaler.fit_transform(df_dummy), columns = df_dummy.columns)
scaled_dataset.shape

(23171, 103)

In [12]:
scaled_dataset.describe()
scaled_dataset.columns[:50]

Index(['age', 'demogweight', 'sex', 'capital-gain', 'capital-loss',
       'hours-per-week', 'income', 'workclass_Federal-gov',
       'workclass_Local-gov', 'workclass_Private', 'workclass_Self-emp-inc',
       'workclass_Self-emp-not-inc', 'workclass_State-gov',
       'workclass_Without-pay', 'education_10th', 'education_11th',
       'education_12th', 'education_1st-4th', 'education_5th-6th',
       'education_7th-8th', 'education_9th', 'education_Assoc-acdm',
       'education_Assoc-voc', 'education_Bachelors', 'education_Doctorate',
       'education_HS-grad', 'education_Masters', 'education_Preschool',
       'education_Prof-school', 'education_Some-college',
       'marital-status_Divorced', 'marital-status_Married-AF-spouse',
       'marital-status_Married-civ-spouse',
       'marital-status_Married-spouse-absent', 'marital-status_Never-married',
       'marital-status_Separated', 'marital-status_Widowed',
       'occupation_Adm-clerical', 'occupation_Armed-Forces',
       'oc

In [13]:
# set predictors and response Y without ? columns
y = scaled_dataset.loc[:,'income']
X = scaled_dataset.loc[:, ~ scaled_dataset.columns.isin(['income'])]

In [14]:
#set training and testing set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1)

In [15]:
## fit model
#! pip install keras
from keras.models import Sequential
from keras.layers import Dense
import numpy as np
np.random.seed(1)

Manually tried: All accuracy is around 85%
1. when epochs = 50, batch_size = 25:
  **i.** one hidden layer with # of nodes = 10, 80, 103, 200, 1000
   **ii.** 5 hidden layers 
2. when epochs = 100, batch_size = 100:
  **i.** one hidden layer with # of nodes = 10, 80, 200
3. https://stats.stackexchange.com/questions/181/how-to-choose-the-number-of-hidden-layers-and-nodes-in-a-feedforward-neural-netw -> one hidden layer is sufficient

#### Hyper Parameters: Kernel Initializer, Learning rate, Hidden Layers, Batch size, (Epochs), Loss Function, Activation Function


In [16]:
model1 = Sequential()
# Set hidden layers
# Dense(# of nodes for this hidden layer, input_dim = p)
model1.add(Dense(30, input_dim = 102, kernel_initializer = 'uniform', activation = 'sigmoid')) # 1st hidden layer
# Set a output layer
model1.add(Dense(1, kernel_initializer = 'uniform', activation = 'sigmoid'))
# combine layers together
model1.compile(loss = 'binary_crossentropy', optimizer = 'rmsprop', metrics = ['accuracy'])

In [17]:
model1.fit(X_train, y_train, epochs = 100, batch_size = 100, verbose = 2)

Epoch 1/100
186/186 - 1s - loss: 0.5499 - accuracy: 0.7506 - 1s/epoch - 7ms/step
Epoch 2/100
186/186 - 0s - loss: 0.4615 - accuracy: 0.7528 - 311ms/epoch - 2ms/step
Epoch 3/100
186/186 - 0s - loss: 0.4041 - accuracy: 0.7882 - 282ms/epoch - 2ms/step
Epoch 4/100
186/186 - 0s - loss: 0.3811 - accuracy: 0.8242 - 297ms/epoch - 2ms/step
Epoch 5/100
186/186 - 0s - loss: 0.3708 - accuracy: 0.8278 - 291ms/epoch - 2ms/step
Epoch 6/100
186/186 - 0s - loss: 0.3648 - accuracy: 0.8286 - 285ms/epoch - 2ms/step
Epoch 7/100
186/186 - 0s - loss: 0.3607 - accuracy: 0.8301 - 283ms/epoch - 2ms/step
Epoch 8/100
186/186 - 0s - loss: 0.3576 - accuracy: 0.8312 - 281ms/epoch - 2ms/step
Epoch 9/100
186/186 - 0s - loss: 0.3551 - accuracy: 0.8317 - 303ms/epoch - 2ms/step
Epoch 10/100
186/186 - 0s - loss: 0.3531 - accuracy: 0.8328 - 282ms/epoch - 2ms/step
Epoch 11/100
186/186 - 0s - loss: 0.3512 - accuracy: 0.8334 - 297ms/epoch - 2ms/step
Epoch 12/100
186/186 - 0s - loss: 0.3497 - accuracy: 0.8341 - 287ms/epoch - 2

In [18]:
loss, accuracy = model1.evaluate(X_test, y_test)
accuracy

145/145 [==============================] - 0s 2ms/step - loss: 0.3352 - accuracy: 0.8475


0.847464919090271

### Grid Search to find best hyperparameters

In [19]:
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
def ANN_model(neurons = 1):
  model = Sequential()
  model.add(Dense(neurons, input_dim = 103, kernel_initializer = 'uniform', activation = 'sigmoid'))
  model.add(Dense(1, kernel_initializer = 'uniform', activation = 'sigmoid'))
  model.compile(loss = 'binary_crossentropy', optimizer = 'rmsprop', metrics = ['accuracy'])
  return model
model2 = KerasClassifier(build_fn = ANN_model, epochs = 100, batch_size = 100, verbose = 2)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  if __name__ == '__main__':


In [ ]:
nodes = [10,30,50,80]
parameter_grid = dict(neurons = nodes)
grid = GridSearchCV(estimator = model2, param_grid = parameter_grid, cv = 10)
grid_result = grid.fit(X,y)

Epoch 1/100
Epoch 1/100
Epoch 1/100
Epoch 1/100
Epoch 1/100
Epoch 1/100
Epoch 1/100
Epoch 1/100
Epoch 1/100
Epoch 1/100
Epoch 1/100
Epoch 1/100
Epoch 1/100


In [ ]:
print('Best accuracy: %f with %s' % (grid_result.best_score_, grid_result.best_params_))

In [ ]:
MSE = grid_result.cv_results_['mean_test_score']
MSE

# neural graph

In [ ]:
# visualize ANN
! pip3 install ann_visualizer
! pip install graphviz
from ann_visualizer.visualize import ann_viz;
ann_viz(model1, title= "ANN 1")

#confusion matrix

In [ ]:
# confusion matrix
! pip install dmba
from dmba import classificationSummary

In [ ]:
# make prediction for both training and validate and set 0.5 as threshold accroding to Bayes Thm
prediction_train = model1.predict(X_train)
prediction_01_train = (model1.predict(X_train) > 0.5).astype(int)
# confusion matrix for training
classificationSummary(y_train, prediction_01_train)

prediction_test = model1.predict(X_test)
prediction_01_test = (model1.predict(X_test) > 0.5).astype(int)
# confusion matrix for training
classificationSummary(y_test, prediction_01_test)

# feature importance

In [ ]:
# shap method for feature importance
! pip install lime
! pip install eli5
! pip install shap

In [ ]:
import shap
#import eli5
#from eli5.sklearn import PermutationImportance
#import lime
#import lime.lime_tabular


explainer = shap.KernelExplainer(model1, X_train)
shap_values = explainer.shap_values(X_train)
#shap.force_plot(explainer.expected_value, shap_values[0,:], X.iloc[0,:])
shap.summary_plot(shap_values, X_train, plot_type = "bar")

In [ ]:
import lime.lime_tabular
explainer = lime.lime_tabular.LimeTabularExplainer(X_train, feature_names=list(X), class_names=[0, 1], mode ='classification')
# added top_labels=1
exp = explainer.explain_instance(X_test[2], model1.predict, num_features = 103, top_labels=1)

In [ ]:
# feature importance - by mean_weights
col = X.columns.tolist() # predictos' name
predictor_weights = model1.get_weights()
mean_weights = [np.abs(i.mean()) for i in predictor_weights[0]] # mean weight for each predictor, list
scaled_mean_weight = scaler.fit_transform(pd.DataFrame(mean_weights)) # scale weights between 0 and 1
predictors = pd.DataFrame({'Predictors': col})
predictors['mean_weight'] = scaled_mean_weight
predictors_order_weight = predictors.sort_values(['mean_weight'], ascending = False)
predictors_order_weight.head(10)

In [ ]:
# e.
pred_df = df.loc[:, ~ df.columns.isin(['income'])]
pred_df['pred_income'] = pd.DataFrame(prediction_01)
over50 = pred_df[pred_df['pred_income'] == 1]
over50['occupation'].value_counts().to_frame()
over50['education'].value_counts().to_frame()
over50['age'].value_counts().to_frame().head(20)

In [ ]:
# plot for 'occupation'
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
sns.catplot(data = pred_df, x = 'age', hue = 'pred_income', kind = 'count', height = 5, aspect = 3)
plt.xticks(rotation=45)

In [ ]:
#f.
# important 'capital-gain'
scaled_CG = scaler.fit_transform(pd.DataFrame(pred_df['capital-gain'])
pred
sns.histplot(data = pred_df, x = 'scaled_CG')

#part 2

Import testing data

In [20]:
test_df = pd.read_csv('USCensusTest.csv')
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7561 entries, 0 to 7560
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             7561 non-null   int64 
 1   workclass       7561 non-null   object
 2   demogweight     7561 non-null   int64 
 3   education       7561 non-null   object
 4   education-num   7561 non-null   int64 
 5   marital-status  7561 non-null   object
 6   occupation      7561 non-null   object
 7   relationship    7561 non-null   object
 8   race            7561 non-null   object
 9   sex             7561 non-null   object
 10  capital-gain    7561 non-null   int64 
 11  capital-loss    7561 non-null   int64 
 12  hours-per-week  7561 non-null   int64 
 13  native-country  7561 non-null   object
dtypes: int64(6), object(8)
memory usage: 827.1+ KB


In [21]:
no_edunum_test_df = test_df.drop(columns = ['education-num'])
cat_var = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'native-country']
test_df_dummy = pd.get_dummies(no_edunum_test_df, columns = cat_var)
# set binary values for 'sex' and 'income'
test_df_dummy['sex'] = test_df_dummy['sex'].replace('Male', 1)
test_df_dummy['sex'] = test_df_dummy['sex'].replace('Female', 0)
test_df_dummy

,age,demogweight,sex,capital-gain,capital-loss,hours-per-week,workclass_?,workclass_Federal-gov,workclass_Local-gov,workclass_Never-worked,workclass_Private,workclass_Self-emp-inc,workclass_Self-emp-not-inc,workclass_State-gov,workclass_Without-pay,education_10th,education_11th,education_12th,education_1st-4th,education_5th-6th,education_7th-8th,education_9th,education_Assoc-acdm,education_Assoc-voc,education_Bachelors,education_Doctorate,education_HS-grad,education_Masters,education_Preschool,education_Prof-school,education_Some-college,marital-status_Divorced,marital-status_Married-AF-spouse,marital-status_Married-civ-spouse,marital-status_Married-spouse-absent,marital-status_Never-married,marital-status_Separated,marital-status_Widowed,occupation_?,occupation_Adm-clerical,occupation_Armed-Forces,occupation_Craft-repair,occupation_Exec-managerial,occupation_Farming-fishing,occupation_Handlers-cleaners,occupation_Machine-op-inspct,occupation_Other-service,occupation_Priv-house-serv,occupation_Prof-specialty,occupation_Protective-serv,occupation_Sales,occupation_Tech-support,occupation_Transport-moving,relationship_Husband,relationship_Not-in-family,relationship_Other-relative,relationship_Own-child,relationship_Unmarried,relationship_Wife,race_Amer-Indian-Eskimo,race_Asian-Pac-Islander,race_Black,race_Other,race_White,native-country_?,native-country_Cambodia,native-country_Canada,native-country_China,native-country_Columbia,native-country_Cuba,native-country_Dominican-Republic,native-country_Ecuador,native-country_El-Salvador,native-country_England,native-country_France,native-country_Germany,native-country_Greece,native-country_Guatemala,native-country_Haiti,native-country_Honduras,native-country_Hong,native-country_Hungary,native-country_India,native-country_Iran,native-country_Ireland,native-country_Italy,native-country_Jamaica,native-country_Japan,native-country_Laos,native-country_Mexico,native-country_Nicaragua,native-country_Outlying-US(Guam-USVI-etc),native-country_Peru,native-country_Philippines,native-country_Poland,native-country_Portugal,native-country_Puerto-Rico,native-country_Scotland,native-country_South,native-country_Taiwan,native-country_Thailand,native-country_Trinadad&Tobago,native-country_United-States,native-country_Vietnam,native-country_Yugoslavia
0,44,162028,0,0,0,44,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1,20,197997,1,0,0,40,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,46,98350,1,0,0,37,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,39,165848,1,0,0,50,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4,34,178615,1,0,0,40,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7556,27,257302,0,0,0,38,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [22]:
test_scaled_df = pd.DataFrame(scaler.fit_transform(test_df_dummy), columns = test_df_dummy.columns)
test_scaled_df.head()

,age,demogweight,sex,capital-gain,capital-loss,hours-per-week,workclass_?,workclass_Federal-gov,workclass_Local-gov,workclass_Never-worked,workclass_Private,workclass_Self-emp-inc,workclass_Self-emp-not-inc,workclass_State-gov,workclass_Without-pay,education_10th,education_11th,education_12th,education_1st-4th,education_5th-6th,education_7th-8th,education_9th,education_Assoc-acdm,education_Assoc-voc,education_Bachelors,education_Doctorate,education_HS-grad,education_Masters,education_Preschool,education_Prof-school,education_Some-college,marital-status_Divorced,marital-status_Married-AF-spouse,marital-status_Married-civ-spouse,marital-status_Married-spouse-absent,marital-status_Never-married,marital-status_Separated,marital-status_Widowed,occupation_?,occupation_Adm-clerical,occupation_Armed-Forces,occupation_Craft-repair,occupation_Exec-managerial,occupation_Farming-fishing,occupation_Handlers-cleaners,occupation_Machine-op-inspct,occupation_Other-service,occupation_Priv-house-serv,occupation_Prof-specialty,occupation_Protective-serv,occupation_Sales,occupation_Tech-support,occupation_Transport-moving,relationship_Husband,relationship_Not-in-family,relationship_Other-relative,relationship_Own-child,relationship_Unmarried,relationship_Wife,race_Amer-Indian-Eskimo,race_Asian-Pac-Islander,race_Black,race_Other,race_White,native-country_?,native-country_Cambodia,native-country_Canada,native-country_China,native-country_Columbia,native-country_Cuba,native-country_Dominican-Republic,native-country_Ecuador,native-country_El-Salvador,native-country_England,native-country_France,native-country_Germany,native-country_Greece,native-country_Guatemala,native-country_Haiti,native-country_Honduras,native-country_Hong,native-country_Hungary,native-country_India,native-country_Iran,native-country_Ireland,native-country_Italy,native-country_Jamaica,native-country_Japan,native-country_Laos,native-country_Mexico,native-country_Nicaragua,native-country_Outlying-US(Guam-USVI-etc),native-country_Peru,native-country_Philippines,native-country_Poland,native-country_Portugal,native-country_Puerto-Rico,native-country_Scotland,native-country_South,native-country_Taiwan,native-country_Thailand,native-country_Trinadad&Tobago,native-country_United-States,native-country_Vietnam,native-country_Yugoslavia
0,0.369863,0.129191,0.0,0.0,0.0,0.438776,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,0.041096,0.160534,1.0,0.0,0.0,0.397959,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0.397260,0.073703,1.0,0.0,0.0,0.367347,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.301370,0.132520,1.0,0.0,0.0,0.500000,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.

In [23]:
test_X = test_scaled_df.loc[:, ~ test_scaled_df.columns.isin(['workclass_?', 'occupation_?', 'native-country_?'])]
test_X

,age,demogweight,sex,capital-gain,capital-loss,hours-per-week,workclass_Federal-gov,workclass_Local-gov,workclass_Never-worked,workclass_Private,workclass_Self-emp-inc,workclass_Self-emp-not-inc,workclass_State-gov,workclass_Without-pay,education_10th,education_11th,education_12th,education_1st-4th,education_5th-6th,education_7th-8th,education_9th,education_Assoc-acdm,education_Assoc-voc,education_Bachelors,education_Doctorate,education_HS-grad,education_Masters,education_Preschool,education_Prof-school,education_Some-college,marital-status_Divorced,marital-status_Married-AF-spouse,marital-status_Married-civ-spouse,marital-status_Married-spouse-absent,marital-status_Never-married,marital-status_Separated,marital-status_Widowed,occupation_Adm-clerical,occupation_Armed-Forces,occupation_Craft-repair,occupation_Exec-managerial,occupation_Farming-fishing,occupation_Handlers-cleaners,occupation_Machine-op-inspct,occupation_Other-service,occupation_Priv-house-serv,occupation_Prof-specialty,occupation_Protective-serv,occupation_Sales,occupation_Tech-support,occupation_Transport-moving,relationship_Husband,relationship_Not-in-family,relationship_Other-relative,relationship_Own-child,relationship_Unmarried,relationship_Wife,race_Amer-Indian-Eskimo,race_Asian-Pac-Islander,race_Black,race_Other,race_White,native-country_Cambodia,native-country_Canada,native-country_China,native-country_Columbia,native-country_Cuba,native-country_Dominican-Republic,native-country_Ecuador,native-country_El-Salvador,native-country_England,native-country_France,native-country_Germany,native-country_Greece,native-country_Guatemala,native-country_Haiti,native-country_Honduras,native-country_Hong,native-country_Hungary,native-country_India,native-country_Iran,native-country_Ireland,native-country_Italy,native-country_Jamaica,native-country_Japan,native-country_Laos,native-country_Mexico,native-country_Nicaragua,native-country_Outlying-US(Guam-USVI-etc),native-country_Peru,native-country_Philippines,native-country_Poland,native-country_Portugal,native-country_Puerto-Rico,native-country_Scotland,native-country_South,native-country_Taiwan,native-country_Thailand,native-country_Trinadad&Tobago,native-country_United-States,native-country_Vietnam,native-country_Yugoslavia
0,0.369863,0.129191,0.0,0.000000,0.0,0.438776,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,0.041096,0.160534,1.0,0.000000,0.0,0.397959,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0.397260,0.073703,1.0,0.000000,0.0,0.367347,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.301370,0.132520,1.0,0.000000,0.0,0.500000,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.2328

### Grid Search Optimization

In [27]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.models import load_model


#empty model
classifier = Sequential()

classifier.add(Dense(output_dim=15, init="uniform", activation='softmax',input_dim = 15))
classifier.add(Dropout(rate=0.1))

TypeError: ignored

In [24]:
nodes = list(range(25, 35, 5))
Batch_size = list(range(10, 101, 10))
epochs = list(range(50,65, 5))
activations = ["relu", "sigmoid", "softmax"]


In [24]:
def ANN_model_relu(unit):
  # creating the layers of the NN
  ann = Sequential()
  ann.add(Dense(units=unit, activation='relu'))
  ann.add(Dense(units=1, activation='relu'))
  ann.compile(optimizer = 'rmsprop', loss = 'binary_crossentropy', metrics = ['accuracy'])
  return ann

params={ 
    'unit':[30, 40, 50, 60], 
    'batch_size':[10, 20, 30, 40, 50, 60, 70, 80, 90, 100],
		'epochs':[45, 50, 55, 60, 65]
}

# optimized_model = KerasClassifier(build_fn = ANN_model_relu(), epochs = epochs, batch_size=Batch_size, verbose = 2)
  
optimized_model=KerasClassifier(build_fn=ANN_model_relu)
gs=GridSearchCV(estimator=optimized_model, param_grid=params, cv=10)
gs = gs.fit(X, y)

loss, accuracy = optimized_model.evaluate(X_test, y_test)
accuracy

print('Best accuracy: %f with %s' % (grid_result.best_score_, grid_result.best_params_))
MSE = grid_result.cv_results_['mean_test_score']
MSE

NameError: ignored

In [28]:
def ANN_model_sigmoid(unit):
  # creating the layers of the NN
  ann = Sequential()
  ann.add(Dense(units=unit, activation='sigmoid'))
  ann.add(Dense(units=1, activation='sigmoid'))
  ann.compile(optimizer = 'rmsprop', loss = 'binary_crossentropy', metrics = ['accuracy'])
  return ann

params={ 
    'unit':[30, 40, 50, 60], 
    'batch_size':[10, 20, 30, 40, 50, 60, 70, 80, 90, 100],
		'epochs':[45, 50, 55, 60, 65]
}

# optimized_model = KerasClassifier(build_fn = ANN_model_relu(), epochs = epochs, batch_size=Batch_size, verbose = 2)
  
optimized_model=KerasClassifier(build_fn=ANN_model_sigmoid)
gs=GridSearchCV(estimator=optimized_model, param_grid=params, cv=10)
gs = gs.fit(X, y)

loss, accuracy = optimized_model.evaluate(X_test, y_test)
accuracy

print('Best accuracy: %f with %s' % (grid_result.best_score_, grid_result.best_params_))
MSE = grid_result.cv_results_['mean_test_score']
MSE

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.


Epoch 1/45
2086/2086 [==============================] - 9s 4ms/step - loss: 0.3999 - accuracy: 0.8147
Epoch 2/45
2086/2086 [==============================] - 8s 4ms/step - loss: 0.3573 - accuracy: 0.8333
Epoch 3/45
2086/2086 [==============================] - 10s 5ms/step - loss: 0.3522 - accuracy: 0.8357
Epoch 4/45
2086/2086 [==============================] - 5s 2ms/step - loss: 0.3506 - accuracy: 0.8370
Epoch 5/45
2086/2086 [==============================] - 4s 2ms/step - loss: 0.3495 - accuracy: 0.8386
Epoch 6/45
2086/2086 [==============================] - 4s 2ms/step - loss: 0.3488 - accuracy: 0.8397
Epoch 7/45
2086/2086 [==============================] - 4s 2ms/step - loss: 0.3483 - accuracy: 0.8409
Epoch 8/45
2086/2086 [==============================] - 4s 2ms/step - loss: 0.3478 - accuracy: 0.8401
Epoch 9/45
2086/2086 [==============================] - 4s 2ms/step - loss: 0.3473 - accuracy: 0.8421
Epoch 10/45
2086/2086 [==============================] - 4s 2ms/step - loss: 0.34

KeyboardInterrupt: ignored